# Sistema de Informação de Agravos de Notificação (SINAN)

Esse notebook tem como objetivo realizar o ELT dos dados provenientes do SINAN que serão utilizados no projeto Dengue.

### Libs

In [1]:
%run ../config/bootstrap.py

In [2]:
import pandas as pd
from datetime import datetime 
from utils import get_project_root, save_parquet_in_chunks  
import basedosdados as bd

In [3]:
project_root = get_project_root() 
billing_id = 'trabalho-pratico-ml'

Para fazer login:
``` shell
gcloud auth application-default login
```

In [4]:
%load_ext sql
%reload_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.displaylimit = 100

# %sql bigquery://genuine-orb-470122-v0
%sql bigquery://trabalho-pratico-ml

E0000 00:00:1763427591.888445  137099 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Connecting to 'bigquery://trabalho-pratico-ml'

In [17]:
%%sql
SELECT * FROM `basedosdados.br_ms_sinan.dicionario` limit 10

Running query in 'bigquery://trabalho-pratico-ml'

10 rows affected.

,id_tabela,nome_coluna,chave,cobertura_temporal,valor
0,microdados_dengue,escolaridade_paciente,1,(1),1ª a 4ª série incompleta do EF
1,microdados_dengue,escolaridade_paciente,2,(1),4ª série completa do EF ( antigo 1° grau)
2,microdados_dengue,escolaridade_paciente,3,(1),5ª à 8ª série incompleta do EF (antigo ginásio ou 1° grau)
3,microdados_dengue,raca_cor_paciente,3,(1),Amarela
4,microdados_dengue,idade_paciente,4,(1),Ano
5,microdados_dengue,raca_cor_paciente,1,(1),Branca
6,microdados_dengue,criterio_confirmacao,2,(1),Clínico Epidemiológico
7,microdados_dengue,evolucao_caso,1,(1),Cura
8,microdados_dengue,idade_paciente,2,(1),Dia
9,microdados_dengue,escolaridade_paciente,8,(1),Educação superior completa


### Testing

In [ ]:
%%sql  
SELECT ano, count(*) as qt FROM `basedosdados.br_ms_sinan.microdados_dengue` group by 1 order by 1 

# bi_query

### low_risk

In [ ]:
%%sql df <<
SELECT
    sigla_uf_residencia,
    idade_paciente,
    sexo_paciente,
    raca_cor_paciente,
    gestante_paciente,
    possui_doenca_autoimune,
    possui_diabetes,
    possui_doencas_hematologicas,
    possui_hepatopatias,
    possui_doenca_renal,
    possui_hipertensao,
    possui_doenca_acido_peptica,
    apresenta_febre,
    apresenta_cefaleia,
    apresenta_exantema,
    apresenta_dor_costas,
    apresenta_mialgia,
    apresenta_vomito,
    apresenta_conjutivite,
    apresenta_dor_retroorbital,
    apresenta_artralgia,
    apresenta_artrite,
    apresenta_leucopenia,
    apresenta_petequias,
    prova_laco,
    classificacao_final,
    evolucao_caso,
    DATE_DIFF(data_notificacao, data_primeiros_sintomas, DAY) AS dias_sintomas_notificacao
FROM
    `basedosdados.br_ms_sinan.microdados_dengue`
WHERE
    classificacao_final = '10'
    AND evolucao_caso != '2'
    AND evolucao_caso IS NOT NULL
    AND data_primeiros_sintomas IS NOT NULL
    AND data_notificacao IS NOT NULL
    AND ano = 2021

In [ ]:
df.info()

In [ ]:
save_parquet_in_chunks(df, "data/2_silver/2021_low_risk")

## alarm + severe

In [ ]:
%%sql df <<
SELECT
    sigla_uf_residencia,
    idade_paciente,
    sexo_paciente,
    raca_cor_paciente,
    gestante_paciente,
    possui_doenca_autoimune,
    possui_diabetes,
    possui_doencas_hematologicas,
    possui_hepatopatias,
    possui_doenca_renal,
    possui_hipertensao,
    possui_doenca_acido_peptica,
    apresenta_febre,
    apresenta_cefaleia,
    apresenta_exantema,
    apresenta_dor_costas,
    apresenta_mialgia,
    apresenta_vomito,
    apresenta_conjutivite,
    apresenta_dor_retroorbital,
    apresenta_artralgia,
    apresenta_artrite,
    apresenta_leucopenia,
    apresenta_petequias,
    prova_laco,
    classificacao_final,
    evolucao_caso,
    DATE_DIFF(data_notificacao, data_primeiros_sintomas, DAY) AS dias_sintomas_notificacao
FROM
    `basedosdados.br_ms_sinan.microdados_dengue`
WHERE
    (classificacao_final IN ('11', '12') OR evolucao_caso = '2')
    AND evolucao_caso IS NOT NULL
    AND classificacao_final IS NOT NULL
    AND data_primeiros_sintomas IS NOT NULL
    AND data_notificacao IS NOT NULL

In [ ]:
df.info()

In [ ]:
save_parquet_in_chunks(df, "data/2_silver/2000-2025_alarm_severe")